### **Packages and dataset load**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm
# import dataframe_image as df___i

color = {"granate":"#BA4A00",
         "amarillo":"#F5B041",
         "verde":"#148F77",
         "blue":"#0051A2",
         "red": "#DD1717"}
color_palette = [color["blue"], 'darkorchid', color['verde'], color['amarillo'],'gray', 'cornflowerblue', color['red']]
sb.set_style('white')

In [3]:
df = pd.read_csv('Data/Dry_Bean_Dataset.csv')

### **Auxiliar functions**

Cross validation

In [4]:
def dataset_creator(models_names: list, columns_names: list, k1: int):
    header = pd.MultiIndex.from_product([models_names, columns_names])
    df = pd.DataFrame(columns=header)
    df['KFold'] = np.arange(1, k1+1)
    df.set_index('KFold', inplace=True)
    return df

def twolevelcv(X: np.array, y: np.array, k1: int, k2: int, models: list, params: dict, rs: int):
    """Allows to compute two level crossvalidation.

    Args:
        X (np.array): Features (numeric)
        y (np.array): Class (objective variable)
        k1 (int): Nº of outer folds
        k2 (int): Nº of inner folds
        models (list): List of models for comparison
        params (dict): Dictionary including the set of parameters. In this case we only tune 1 parameter per model.
        rs (int): Random state
    Returns:
        df: Dataframe
    """
    test_error_dict = {}
    k = 0
    names = [type(m).__name__ for m in models]
    col_names = ['Param. Value', 'Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k += 1
        kf2 = StratifiedKFold(k2, shuffle = True, random_state=rs)
        print(f'Computing KFold {k}/{k1}...')
        # second level split
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            for name, model in zip(names, models):
                if name != 'DummyClassifier':
                    pname = list(params[name].keys())[0]
                    error_test = []
                    for p_ in params[name][pname]:
                        pdict = {pname: p_}
                        model = model.set_params(**pdict)
                        # train the model
                        model.fit(X_train, y_train)
                        # evaluate performance
                        pred2_test = model.predict(X_test)
                        error_test.append(np.sum(pred2_test != y_test)/ y_test.shape[0])
                    min_param = params[name][pname][np.argmin(error_test)]
                else:
                    model.fit(X_train, y_train)
                    pred2_test = model.predict(X_test)
                    error_test = np.sum(pred2_test != y_test)/ y_test.shape[0]
                    min_param = np.NaN
                df.loc(axis = 1)[name, 'Error'][k] = np.min(error_test)
                df.loc(axis = 1)[name, 'Param. Value'][k] = min_param
    return df

# **1 - Regression**

In [5]:
columns = df.columns.values
X = df.drop(columns='Class').values
y = df['roundness']


print('· NUMBER OF FEATURES:', X.shape[1])
print('\n· FEATURES:\n', columns[:-1])
print('\n· NUMBER OF DATA POINTS:', X.shape[0])

· NUMBER OF FEATURES: 16

· FEATURES:
 ['Area' 'Perimeter' 'MajorAxisLength' 'MinorAxisLength' 'AspectRation'
 'Eccentricity' 'ConvexArea' 'EquivDiameter' 'Extent' 'Solidity'
 'roundness' 'Compactness' 'ShapeFactor1' 'ShapeFactor2' 'ShapeFactor3'
 'ShapeFactor4']

· NUMBER OF DATA POINTS: 13611


### **Part A. *Linear regression.***

### **Part B. *Other models. Evaluation.***

# **2 - Classification**

### **Dataset preparation**

In [5]:
# Copying object without editing the original
df_ = df.copy(deep=True)
# Doing this we can choose to use outliers filter or not

In [6]:
columns = df_.columns.values
X = df_.drop(columns='Class').values
y = df_['Class']
le = LabelEncoder()
y_ = le.fit_transform(y)
classes = y.unique()

print('· NUMBER OF FEATURES:', X.shape[1])
print('\n· FEATURES:', columns[:-1])
print('\n· NUMBER OF DATA POINTS:', X.shape[0])
print('\n· CLASSES:', classes)
print('\n· NUMBER OF CLASSES:', len(classes))

· NUMBER OF FEATURES: 16

· FEATURES: ['Area' 'Perimeter' 'MajorAxisLength' 'MinorAxisLength' 'AspectRation'
 'Eccentricity' 'ConvexArea' 'EquivDiameter' 'Extent' 'Solidity'
 'roundness' 'Compactness' 'ShapeFactor1' 'ShapeFactor2' 'ShapeFactor3'
 'ShapeFactor4']

· NUMBER OF DATA POINTS: 13611

· CLASSES: ['SEKER' 'BARBUNYA' 'BOMBAY' 'CALI' 'HOROZ' 'SIRA' 'DERMASON']

· NUMBER OF CLASSES: 7


#### **Transformations**

Outliers removal

In [7]:
Threshold_ = 3
outlier_index = []
df_ = pd.DataFrame(columns=df.columns)
index = 0
for K in classes:
    outlier_index = []
    a = df.loc[df["Class"] == K]
    value = a.drop(columns='Class').values
    for j in range(16):
        std = np.std(value[:, j])
        mean = np.mean(value[:, j])
        for i in range(value[:, j].shape[0]):
            if (value[i, j] - mean) / std > Threshold_:
                outlier_index.append(i + index)
    index = i + index + 1
    outlier_index = np.unique(outlier_index)
    a = a.drop(outlier_index)
    df_ = pd.concat([df_,a])
df_.reset_index(drop=True, inplace=True)
print(f'Filtered outliers: {df.shape[0] - df_.shape[0]}')

Filtered outliers: 298


Standarization

In [8]:
# Standarization of the dataset
sc = StandardScaler()
X_stdz = sc.fit_transform(X)
df_stdz = pd.DataFrame(columns = columns[:-1], data = X_stdz)
df_stdz['Class'] = y_

### **2.2 Logistic regression *vs.* Neural Network *vs.* Baseline**

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier 
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
random_state = 1

#### Logistic regression

In [10]:
# model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [11]:
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# n_scores = cross_val_score(model, X_stdz, y_, scoring = 'accuracy', cv=cv, n_jobs=-1)
# # report the model performance
# print('Mean Accuracy: %.3f (+-%.3f)' % (np.mean(n_scores), np.std(n_scores)))

### **2.3 Cross-Validation table**

In [12]:
params = {}
params['LogisticRegression'] = {'C': [0.03, 0.05, 0.07, 0.09]}
params['DummyClassifier'] = [None]
params['MLPClassifier'] = {'hidden_layer_sizes': [(8,), (16,), (20,)]}
models = [LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state= random_state),
        DummyClassifier(strategy='most_frequent', random_state=random_state),
        MLPClassifier(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000, 
        early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False, learning_rate ='adaptive', learning_rate_init=0.01)]
k1 = 10
k2 = 10
Table = twolevelcv(X = X_stdz, y = y_, k1 = k1, k2 = k2, models = models, params = params, rs = random_state)

Computing KFold 1/10...


100%|██████████| 10/10 [00:26<00:00,  2.62s/it]


Computing KFold 2/10...


100%|██████████| 10/10 [00:24<00:00,  2.47s/it]


Computing KFold 3/10...


...

100%|██████████| 10/10 [00:24<00:00,  2.47s/it]


Computing KFold 9/10...


100%|██████████| 10/10 [00:25<00:00,  2.51s/it]


Computing KFold 10/10...


100%|██████████| 10/10 [00:25<00:00,  2.53s/it]


In [13]:
Table

LogisticRegression           DummyClassifier           MLPClassifier  \
            Param. Value     Error    Param. Value     Error  Param. Value   
KFold                                                                        
1                   0.09  0.042484             NaN  0.788399         (16,)   
2                   0.09  0.036735             NaN   0.78449          (8,)   
3                   0.09  0.036735             NaN   0.78449          (8,)   
4                   0.09  0.036735             NaN   0.78449          (8,)   
5                   0.09  0.036735             NaN   0.78449          (8,)   
6                   0.07  0.037551             NaN  0.790204          (8,)   
7                   0.07  0.037551             NaN  0.790204          (8,)   
8                   0.07  0.034286             NaN  0.790204          (8,)   
9                   0.07  0.034286             NaN  0.790204          (8,)   
10                  0.07  0.044082             NaN  0.788571          (8,)   

                 
          Error  
KFold            
1      0.039216  
2      0.035102  
3      0.034286  
4      0.035918  
5      0.035918  
6      0.040816  
7          0.04  
8      0.041633  
9      0.042449  
10     0.044082

In [18]:
Table = Table.round(3)
Table.to_csv(r'Results\Table_classification3.csv')

### **2.4 Stadistical Evaluation**

In [14]:
from itertools import combinations

def McNemar(models: list, X: np.array, y: np.array, k1: int, rs: int):
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    k = 0
    # setting up all the possible combinations between the different models
    matrix = dict.fromkeys(combinations(range(len(models)), 2))
    for train_idx, test_idx in kf1.split(X, y):
        test_size = test_idx.shape[0]
        yABC = np.empty(shape=(len(models), test_size))
        for i, model in enumerate(models):
            model.fit(X[train_idx,:], y[train_idx])
            y_pred = model.predict(X[test_idx, :])
            yABC[i, :] = 1*(y_pred == y[test_idx])
        for j in list(matrix.keys()):
            if k == 0:
                matrix[j] = np.empty(shape=(k1, 4))
            n11 = np.sum(yABC[j[0],:]*yABC[j[1],:])
            n12 = np.sum(yABC[j[0],:]*(1-yABC[j[1],:]))
            n21 = np.sum(yABC[j[1],:]*(1-yABC[j[0],:]))
            n22 = np.sum((1-yABC[0,:])*(1-yABC[1,:]))
            matrix[j][k] = np.array([n11, n12, n21, n22])
        k+=1
    return matrix

In [18]:
random_state = 1
models = [LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state= random_state, C = 0.07),
        DummyClassifier(strategy='most_frequent', random_state=random_state),
        MLPClassifier(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000, hidden_layer_sizes=(8, ),
        early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False, learning_rate ='adaptive', learning_rate_init=0.01)]
k1 = 10
m = McNemar(models, X_stdz, y_, k1, rs = random_state)

We create the matrix with shape (3: number of models, 10: nº of k-folds, 4: matrix shape)

The matrix is squeezed so we have:
$$\begin{bmatrix}
 n_{11}    & n_{12}  \\
 n_{21}    & n_{22}
\end{bmatrix}$$

Here is $[ n_{11}, n_{12}, n_{21}, n_{22} ]$

$H_0:$ Model A has the same performarnce as model B 

$H_1:$ Model A and model B has different performance

small p_value-> we discard H0 -> Model A and Model B have different performance

### **LOGISTIC REGRESSION *vs*. BASELINE**

In [64]:
combs = list(combinations(range(len(models)), 2))

In [77]:
from scipy.stats import binom
pv_dict = dict.fromkeys(combs)
for j in combs:
    vals = m[j][:, 1:3]
    pv_dict[j] = [binom.cdf(min(vals[i]), n = sum(vals[i]), p = 1/2) for i in range(len(vals))]

Here we get all the p-values from the McNemar's test from all the possible models combinations.


In [1]:
pv_dict

NameError: name 'pv_dict' is not defined

### **2.5 Train logistic regression model**

So in the fourth exercise do we have to repeat the parameter selection process or can just go ahead with the best parameter selection for each model?